In [1]:
pip install optuna

  Using cached alembic-1.13.2-py3-none-any.whl.metadata (7.4 kB)
  Using cached colorlog-6.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached Mako-1.3.5-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached MarkupSafe-2.1.5-cp312-cp312-win_amd64.whl.metadata (3.1 kB)
Using cached alembic-1.13.2-py3-none-any.whl (232 kB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   --------------- ------------------------ 0.8/2.1 MB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 3.3 MB/s eta 0:00:00
Using cached colorlog-6.8.2-py3-none-any.whl (11 kB)
Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl (156 kB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
Using cached Mako-1.3.5-py3-none-any.whl (78 kB)
Using cached Mark

In [32]:
import numpy as np
import pandas as pd

#Service modules
import timeit
import inspect, os

#Import tools to work with data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#Importing metrics 
from sklearn.metrics import roc_auc_score

# Importing ML models 
from catboost import CatBoostClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier

In [33]:
file_path = os.getcwd()
dir_name = os.path.split(file_path)[0]
file_path

'c:\\Users\\User\\OneDrive\\Документы\\Python\\Hackatone\\Models'

In [34]:
FCh_MoFP_path = f"{dir_name}\\database\\train_rdkit_morgan_smote.csv"
FCh_path = f"{dir_name}\\database\\train_rdkit_smote.csv"

In [35]:
df_train_FCh_descr = pd.read_csv(FCh_path)
df_train_FCh_MoFP_descr = pd.read_csv(FCh_MoFP_path, index_col=0)

In [36]:
df_train_FCh_descr.head()

,Y,property,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,1,1,11.824743,11.824743,0.170602,-0.170602,0.269289,14.608696,301.345,286.225,...,0,0,0,0,0,0,0,0,0,0
1,0,1,14.431073,14.431073,0.128988,-0.378575,0.184582,12.480000,646.614,628.470,...,0,0,0,0,0,0,0,0,0,0
2,1,1,10.422197,10.422197,0.248657,-0.670278,0.103849,7.727273,157.133,150.077,...,0,0,0,0,0,0,0,0,0,0
3,1,1,10.521759,10.521759,0.278148,-0.752407,0.342148,16.600000,138.086,136.070,...,0,0,0,0,0,0,0,0,0,1
4,1,1,11.230461,11.230461,0.034298,-0.976244,0.241838,15.789474,264.285,248.157,...,0,0,0,0,0,0,0,0,1,0


In [37]:
df_train_FCh_MoFP_descr.head()

,Y,property,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,...,Bit_1014,Bit_1015,Bit_1016,Bit_1017,Bit_1018,Bit_1019,Bit_1020,Bit_1021,Bit_1022,Bit_1023
0,1,1,11.824743,11.824743,0.170602,-0.170602,0.269289,14.608696,301.345,286.225,...,0,0,0,0,0,0,0,0,0,0
1,0,1,14.431073,14.431073,0.128988,-0.378575,0.184582,12.480000,646.614,628.470,...,0,0,0,0,0,0,0,0,0,0
2,1,1,10.422197,10.422197,0.248657,-0.670278,0.103849,7.727273,157.133,150.077,...,0,0,0,0,0,0,0,0,0,0
3,1,1,10.521759,10.521759,0.278148,-0.752407,0.342148,16.600000,138.086,136.070,...,0,0,0,0,0,0,0,0,0,0
4,1,1,11.230461,11.230461,0.034298,-0.976244,0.241838,15.789474,264.285,248.157,...,0,0,0,0,0,0,0,0,0,0


In [38]:
models = {"XGBoost":XGBClassifier(random_state=42), 
          # "GradientBoosting": GradientBoostingClassifier(random_state=42), 
          # "RandomForest": RandomForestClassifier(random_state=42), 
          # "ExtraTrees": ExtraTreesClassifier(), 
          "LightGBM": LGBMClassifier(verbosity=-1, random_state=42) 
          # "CatBoost": CatBoostClassifier(silent=True, random_state=42)
        }

In [39]:
x_FCh_MoFP = df_train_FCh_MoFP_descr.drop(["Y"], axis=1).copy()
y_FCh_MoFP = df_train_FCh_MoFP_descr["Y"].copy()

x_FCh = df_train_FCh_descr.drop(["Y"], axis=1).copy()
y_FCh = df_train_FCh_descr["Y"].copy()


In [40]:
datas = {"Phys-chem and Morgan desc": {"X": x_FCh_MoFP, "y": y_FCh_MoFP, "file": FCh_MoFP_path}, 
         "Phys-chem desc": {"X": x_FCh, "y": y_FCh, "file": FCh_path}}

In [41]:
results = list()
count = 0
data_num = len(datas.keys())
model_num = len(models.keys())
for data_name, X_y in datas.items():
    scaler = StandardScaler()
    X_y["X"] = pd.DataFrame(scaler.fit_transform(X_y["X"]), columns=X_y["X"].columns)
    X_train, X_test, y_train, y_test = train_test_split(X_y["X"], X_y["y"], test_size=0.2, random_state=42)
    model_res = list()
    for name, model in models.items():
        mod = model
        start = timeit.default_timer()
        mod.fit(X_train, y_train)
        y_pred = model.predict_proba(X_test)[:, 1]
        auc_roc = round(roc_auc_score(y_test, y_pred), 4)
        stop = timeit.default_timer()
        time = round(stop-start, 3)
        model_res.extend([auc_roc])
        count += 1
        print(f" {count}/{data_num*model_num}. Finished model {name} with dataset {data_name}, AUC_ROC: {round(auc_roc, 5)}, spended time: {round(stop-start, 2)} sec")
    results.append(model_res)

 1/4. Finished model XGBoost with dataset Phys-chem and Morgan desc, AUC_ROC: 0.9313, spended time: 8.3 sec
 2/4. Finished model LightGBM with dataset Phys-chem and Morgan desc, AUC_ROC: 0.9308, spended time: 4.13 sec
 3/4. Finished model XGBoost with dataset Phys-chem desc, AUC_ROC: 0.9307, spended time: 3.17 sec
 4/4. Finished model LightGBM with dataset Phys-chem desc, AUC_ROC: 0.9312, spended time: 2.09 sec


In [42]:
models.keys()

dict_keys(['XGBoost', 'LightGBM'])

In [43]:
tup = list()
for name, model in models.items():
    tup.append(tuple([name, "AUC-ROC"]))
    tup.append(tuple([name, "sec"]))
column_index = pd.MultiIndex.from_tuples(tup)

In [44]:
# model_comper = pd.DataFrame(results, columns=column_index)
# model_comper["Descriptors"] = datas.keys()
# model_comper.set_index("Descriptors")

In [45]:
model_comper = pd.DataFrame(results, columns=models.keys())
model_comper["Descriptors"] = datas.keys()
model_comper.set_index("Descriptors", inplace=True)
model_comper

,XGBoost,LightGBM
Descriptors,,
Phys-chem and Morgan desc,0.9313,0.9308
Phys-chem desc,0.9307,0.9312


In [48]:
model_comper.stack().max()

0.9313

In [47]:
# model_AUC_ROC = model_comper.iloc[:, model_comper.columns.get_level_values(1)== "AUC-ROC"].droplevel(level=1, axis=1).copy()

IndexError: Too many levels: Index has only 1 level, not 2

In [41]:
max_model = model_AUC_ROC.max().idxmax()
max_file = model_AUC_ROC[max_model].idxmax()
AUC_ROC_max = model_AUC_ROC.loc[max_file, max_model]

UFuncTypeError: ufunc 'less' did not contain a loop with signature matching types (<class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.StrDType'>) -> None

In [95]:
file = list(model_comper["Descriptors"])[max_file]
file

'Phys-chem desc'

In [99]:
path = datas[file]["file"]

In [110]:
print(f"Максиммальное значение: {AUC_ROC_max},\nдатасет с названием {file},\nс путём '{path}'")

Максиммальное значение: 0.9338,
датасет с названием Phys-chem desc,
с путём 'C:/Users/User/OneDrive/Документы/Python/Hackatone/database/train_rdkit_morgan_smote.csv'
